In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#del_cha = pd.read_csv("/kaggle/input/capacity/chassis.csv")
del_cha = pd.read_csv("/kaggle/input/capacity-jan-mar/final_df_with_min_distance.csv")

In [ ]:
del_cha.head(4)
print(del_cha.shape)

In [ ]:
workshop = pd.read_csv("/kaggle/input/capacity/workshops.csv")

In [ ]:
del_cha.drop(columns='Unnamed: 0',axis=1,inplace =True)

In [ ]:
workshop.drop(columns='Unnamed: 0',axis=1,inplace =True)

In [ ]:
del_cha.head(3)


In [ ]:
del_cha.describe()

In [ ]:
workshop.head(3)

In [ ]:
workshops = pd.read_csv("/kaggle/input/network-expansion/all_region_workshops.csv")

In [ ]:
workshops.head(4)

In [ ]:
workshops.describe()

In [ ]:
workshops = workshops.dropna(subset=['Lattitude'])

In [ ]:
workshops.shape

In [ ]:
valid_latitude_range = (-90.0, 90.0)
valid_longitude_range = (-180.0, 180.0)

workshops['Lattitude'] = workshops['Lattitude'].apply(lambda x: min(max(x, valid_latitude_range[0]), valid_latitude_range[1]))
workshops['Longitude'] = workshops['Longitude'].apply(lambda x: min(max(x, valid_longitude_range[0]), valid_longitude_range[1]))
workshops = workshops[(workshops['Longitude'] != -180) & (workshops['Longitude'] != 180)]

In [ ]:
workshops.describe()

In [ ]:
workshops.info()

In [ ]:
def haversine(lonlat1, lonlat2):
     """
     Calculate the great circle distance between two points
     on the earth (specified in decimal degrees)
     """
     # convert decimal degrees to radians
     lat1, lon1 = lonlat1
     lat2, lon2 = lonlat2
     lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

     #haversine formula
     dlon = lon2 - lon1
     dlat = lat2 - lat1
     a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
     c = 2 * asin(sqrt(a))
     r = 6371  # Radius of earth in kilometers. Use 3956 for miles
     return c * r

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from math import radians, cos, sin, asin, sqrt
from tqdm import tqdm
from math import ceil
import numpy as np
import folium
import warnings
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from scipy import stats as st
from sklearn.cluster import KMeans
from statistics import mode
from geopy.geocoders import Nominatim
from sklearn.cluster import DBSCAN
from haversine import haversine
from haversine import Unit

warnings.filterwarnings('ignore')
geolocator = Nominatim(user_agent="geoapiExercises")

In [ ]:
#for row in tqdm(range(0, del_cha.shape[0]), desc='Calculating dist. b/w chassis'):
    #dis=[]
    #for div in range(0, workshop.shape[0]):
        #d = haversine(del_cha[['Lat', 'Long']].iloc[row],
                      #workshop[['Lattitude', 'Longitude']].iloc[div],
                      #unit ='km')
        #dis.append(d)
    #del_cha.loc[row, 'Min Distance'] = min(dis)

In [ ]:
#del_cha.to_csv("Chassis_with_distance.csv")

In [ ]:
del_cha.columns

In [ ]:
fig = go.Figure(go.Scattermapbox(
    lat = del_cha[del_cha['min_distance'] < 10]['Lat'],
    lon = del_cha[del_cha['min_distance'] < 10]['Long'],
    mode = 'markers',
    marker = dict(color='green', size=15),
    name = "Chassis"))

fig.add_trace(go.Scattermapbox(
    lat = del_cha[del_cha['min_distance'] >= 10]['Lat'],
    lon = del_cha[del_cha['min_distance'] >= 10]['Long'],
    mode = 'markers',
    marker = dict(color='red'),
    name = "Chassis Positions > 10Km"))

# create a new scattermapbox trace for the additional points with a different color and marker size
fig.add_trace(go.Scattermapbox(
    lat = workshops['Lattitude'],
    lon = workshops['Longitude'],
    mode = 'markers',
    marker = dict(color='blue'),
    name = "Service Centers"))

## update the layout of the plot to use a mapbox style and set the center and zoom level
fig.update_layout(mapbox_style='open-street-map', mapbox_center_lon=77, mapbox_center_lat=28, mapbox_zoom=3)
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))

fig.write_html("/kaggle/working/Final_Heatmap.html")

# show the plot
fig.show()